In [1]:
import ast
import csv
from distutils.log import error 
import json
import math
import numpy as np

import led_config_utils
import mesh_config
from funky_lights import wavefront, led_config

from importlib import reload
reload(mesh_config)

START_OFFSET = 0.0
END_OFFSET = 0.0

def createNodesFromCSV(csv_points):
    nodes = None
    prev_node = None
    for point in csv_points:
        node = led_config_utils.Node(p=np.array(point))
        if nodes == None:
            nodes = node
        if prev_node:
            prev_node.next = node
        prev_node = node
    return nodes


all_segments = {}
with open('../config/rad_actual.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # skip header
    segments=[]

    for row in reader:
        uid = int(row[0])
        name = row[1]
        actual_num_leds = int(row[2])
        actual_length = float(row[3])
        reverse = (row[4] == 'TRUE')
        led_offset = int(row[5])
        sub_component = row[6]
        actual_num_adressable_leds = int(row[7])
        csv_points = ast.literal_eval(row[10])

        nodes = createNodesFromCSV(csv_points)
        if nodes == None:
            print('No nodes available.')
            continue
    
        modelled_length = led_config_utils.line_segments_length(nodes)
        modelled_length = modelled_length -  START_OFFSET -  END_OFFSET
        
        leds_distance = modelled_length / actual_num_adressable_leds
        points = led_config_utils.trace_line_segments(nodes, actual_num_adressable_leds, START_OFFSET, leds_distance)
        if len(points) == 0:
            print('No points available.')
            continue

        if reverse:
            points = np.flip(points, axis=0)

        if led_offset > 0:
            points = np.concatenate((points[led_offset:], points[:led_offset]), axis=0)

        segment = led_config.Segment(
            uid=uid, name=name, points=points, num_leds=points.shape[0], length=actual_length)
        print('Segment %s: length=%.1fm, num_leds=%s' % (segment.name, segment.length, segment.num_leds))
        segments.append(segment)
        all_segments[uid] = segment

# Collapse some segments
SEGMENT_1 = [1,11,12,13,14,15]
SEGMENT_2 = [2,21,22,23,24,25,26,27,28,29]
SEGMENT_3 = [3,31,32,33,34,35]
SEGMENT_4 = [4,41,42,43,44,45]

for merge_list in [SEGMENT_1, SEGMENT_2, SEGMENT_3, SEGMENT_4]:
    merged_segment = all_segments[merge_list[0]]
    for uid in merge_list[1:]:
        segment = all_segments.pop(uid)
        merged_segment.merge(segment)

# Create LED config
config = led_config.LedConfig()
for segment in all_segments.values():
    config.led_segments.append(segment)
    config.total_num_segments += 1
    config.total_length += segment.length
    config.total_num_leds += segment.num_leds

with open('../config/led_config_rad.json', 'w', encoding='utf-8') as f:
    json.dump(config.to_dict(), f, ensure_ascii=False, indent=4)



Segment handlebar/vertical_right: length=0.3m, num_leds=21
Segment rack/rack_right: length=0.8m, num_leds=51
Segment rack/rack_left: length=0.8m, num_leds=51
Segment handlebar/vertical_left: length=0.3m, num_leds=21
Segment handlebar/horizontal_left: length=0.1m, num_leds=7
Segment handlebar/horizontal_right: length=0.1m, num_leds=7
Segment back_frame/horizontal_2_right: length=0.4m, num_leds=24
Segment back_frame/vertical_2_right: length=0.4m, num_leds=27
Segment back_frame/horizontal_1_right: length=0.2m, num_leds=13
Segment back_frame/vertical_3_right: length=0.0m, num_leds=3
Segment back_frame/vertical_1_right: length=0.3m, num_leds=20
Segment back_frame/vertical_1_left: length=0.3m, num_leds=20
Segment back_frame/vertical_3_left: length=0.0m, num_leds=3
Segment back_frame/horizontal_1_left: length=0.2m, num_leds=13
Segment back_frame/vertical_2_left: length=0.4m, num_leds=27
Segment back_frame/horizontal_2_left: length=0.4m, num_leds=24
Segment front_frame/vertical_steer_right: le

KeyError: 20